In [1]:
import os
from mltaskfusion.task.vllm import VllmData, VllmModel
from mltaskfusion.db import queue_client, QueueJobModel

os.environ['CONFIG_FILE'] = "./config.conf"

queue_cli = queue_client(queue_name=VllmModel().queue_name)

In [6]:
from mltaskfusion.utils import helper

prompt = """
OCR the text of the image. Extract the text of the following fields and put it in a JSON format: 'why choose us', 'about us', 'company profile'. 

If the 'why choose us', 'about us', 'company profile' fields do not appear in the image, simply return [].

Ignore Other: Ignore all other text information in the picture except for the above fields.

IMPORTANT: 
    1. YOU MUST RESPOND ONLY WITH VALID JSON. DO NOT INCLUDE ANY INTRODUCTION, EXPLANATION, OR EXTRA TEXT. ONLY PROVIDE THE JSON ARRAY.
    2. NO ADDITIONAL TEXT.

Output format:
    [
        {
            "field": "field name 1",
            "sections": [
                {
                    "title": "",
                    "content": "chunk1 text here"
                },
                {
                    "title": "",
                    "content": "chunk2 text here"
                }
            ]
        },
        {
            "field": "field name 2",
            "sections": [
                {
                    "title": "",
                    "content": "chunk3 text here"
                },
                {
                    "title": "",
                    "content": "chunk4 text here"
                }
            ]
        }
    ]                    
"""
data = VllmData(prompt=prompt, image_urls=['http://img.yfisher.com/d0/1723538497787-e9f3d41f-23da-4f41-83c0-427dfa0719a7.png'], max_tokens=1024, id=helper.unique_id())
result = queue_cli.push_and_response(job=QueueJobModel(id=data.id, data=data.model_dump()), seconds=120)

In [3]:
import json
json.loads(result['content'])

[{'field': 'why choose us',
  'sections': [{'title': 'OEM & ODM',
    'content': 'We can customize color ,package,size for the product'},
   {'title': 'Rich Variety',
    'content': 'We have a wide variety for you to choose from'},
   {'title': 'Fast Delivery',
    'content': 'We guarantee that the products will be delivered to you quickly and completely'},
   {'title': '24h Online',
    'content': 'We can online work with you for 24 hours'}]}]